# Compose: Training a model to generate text

In [2]:
import os
import pickle
import numpy as np
#from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from keras.utils import np_utils
from keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape
from keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from keras.layers import Multiply, Lambda, Softmax
import keras.backend as K 
from keras.models import Model
from keras.optimizers import RMSprop

## Set parameters

In [3]:
# run params
section = 'composetxt'
run_id = '0001'
txt_name = 'txtattn'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, txt_name])

store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', txt_name)

if not os.path.exists(run_folder):
    os.mkdir(run_folder)
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))

mode = 'build' # 'load' # 

# data params
intervals = range(1)
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True

## Extract the text

In [4]:
import re

token_type = 'word'

#load in the text and perform some cleanup

seq_length = 20

filename = "./data/aesop/data.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()    
    
#removing text before and after the main stories
start = text.find("THE FOX AND THE GRAPES\n\n\n")
end = text.find("ILLUSTRATIONS\n\n\n[")
text = text[start:end]

start_story = '| ' * seq_length
    
text = start_story + text
text = text.lower()
text = text.replace('\n\n\n\n\n', start_story)
text = text.replace('\n', ' ')
text = re.sub('  +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
text = re.sub('\s{2,}', ' ', text)

len(text)
print(text[:80])

 | | | | | | | | | | | | | | | | | | | | the fox and the grapes . a hungry fox s


In [13]:
from keras.preprocessing.text import Tokenizer

if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)    
    
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0] # tokenized text

print(total_words)

print(str(tokenizer.word_index)[0:50]) # vocabulary {'|': 1, ',': 2, 'the': 3, 'and': 4, '.': 5, 'a': 
print(token_list[:50]) # tokenized text [1, 1, 1, 1, 3, 56, 4, 3, 940,

4170
{'|': 1, ',': 2, 'the': 3, 'and': 4, '.': 5, 'a': 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 56, 4, 3, 940, 5, 6, 382, 56, 94, 77, 216, 1557, 9, 940, 941, 62, 6, 581, 20, 12, 2226, 162, 6, 359, 2227, 2, 4, 158, 11]


In [4]:
# gensim text load, preprocessing, tokenization
from gensim.parsing.preprocessing import preprocess_string
from gensim import corpora

def read_in_chunks(infile, chunk_size=1024*64):
    chunk = infile.read(chunk_size)
    while chunk:
        yield chunk
        chunk = infile.read(chunk_size)
        
def process_data(chunk, text):
    text.append(str(chunk)) # 'utf8' codec can't decode byte 0xc3
    
seq_len = 20
        
filename = "./data/lgtst/Proktols of Neptune.txt"
        
f = open(filename, encoding="utf-8")
text = []
for piece in read_in_chunks(f):
    process_data(piece, text)
    
corpus = preprocess_string(' '.join(text))
dct = corpora.Dictionary([corpus])  # initialize a Dictionary
token_list = [dct.doc2idx(cdoc) for cdoc in [corpus]]

fout = open(filename + 'dct.txt', 'wb') # to be used later by the generator
dct.save_as_text(fout)
fout.close()

## Prepare network I/O

In [5]:
def prepare_sequences(text, n_tokens, seq_len = 32):
    """ Prepare the sequences used to train the Neural Network """

    text_network_input = []
    text_network_output = []

    # create input sequences and the corresponding outputs
    for i in range(len(text) - seq_len): 
        text_network_input.append(text[i:i + seq_len])
        text_network_output.append(text[i + seq_len])

    n_patterns = len(text_network_input)
    # reshape the input into a format compatible with LSTM layers
    text_network_input = np.reshape(text_network_input, (n_patterns, seq_len))
    network_input = [text_network_input]

    text_network_output = np_utils.to_categorical(text_network_output, num_classes=n_tokens)
    network_output = [text_network_output]

    return (network_input, network_output)

In [6]:
#network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)
network_input, network_output = prepare_sequences(token_list[0], len(dct)+1, seq_len)

In [7]:
print('text input')
print(network_input[0])

print('text output')
print(network_output[0])

text input
[[ 940  830  605 ...  983  427 1159]
 [ 830  605  588 ...  427 1159  165]
 [ 605  588 1159 ... 1159  165  680]
 ...
 [1239  757 1287 ...  697 1155  886]
 [ 757 1287  684 ... 1155  886 1096]
 [1287  684 1445 ...  886 1096   60]]
text output
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Create the structure of the neural network

In [8]:
def create_network(n_tokens, embed_size = 100, rnn_units = 256, use_attention = False):
    """ create the structure of the neural network """

    text_in = Input(shape = (None,))

    x1 = Embedding(n_tokens, embed_size)(text_in) 

    #x = Concatenate()([x1,x2])
    x = x1 # todo: remove and change x1

    x = LSTM(rnn_units, return_sequences=True)(x)
    # x = Dropout(0.2)(x)

    if use_attention:

        x = LSTM(rnn_units, return_sequences=True)(x)
        # x = Dropout(0.2)(x)

        e = Dense(1, activation='tanh')(x)
        e = Reshape([-1])(e)
        alpha = Activation('softmax')(e)

        alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha)) # todo: check the 2, 1

        c = Multiply()([x, alpha_repeated])
        c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
    
    else:
        c = LSTM(rnn_units)(x)
        # c = Dropout(0.2)(c)
                                    
    text_out = Dense(n_tokens, activation = 'softmax', name = 'text')(c)
   
    model = Model(text_in, text_out)

    if use_attention:
        att_model = Model(text_in, alpha)
    else:
        att_model = None

    opti = RMSprop(lr = 0.001)
    model.compile(loss=['categorical_crossentropy'], optimizer=opti)

    return model, att_model

In [9]:
#model, att_model = create_network(n_tokens, embed_size, rnn_units, use_attention)
model, att_model = create_network(len(dct)+1, embed_size, rnn_units, use_attention)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    145100      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, None, 256)    365568      embedding_1[0][0]                
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, None, 256)    525312      lstm_1[0][0]                     
____________________________________________________________________________________________

## Train the neural network

In [10]:
weights_folder = os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [11]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=200, batch_size=32 # 200 epochs
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )


D:\Anaconda\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4006 samples, validate on 1002 samples
Epoch 1/200
4006/4006 [==============================] - 8s 2ms/step - loss: 6.9317 - val_loss: 7.0053
Epoch 2/200
4006/4006 [==============================] - 7s 2ms/step - loss: 6.6440 - val_loss: 7.2083
Epoch 3/200
4006/4006 [==============================] - 7s 2ms/step - loss: 6.5436 - val_loss: 7.3089
Epoch 4/200
4006/4006 [==============================] - 7s 2ms/step - loss: 6.4609 - val_loss: 7.1668
Epoch 5/200
4006/4006 [==============================] - 7s 2ms/step - loss: 6.4038 - val_loss: 7.3158
Epoch 6/200
4006/4006 [==============================] - 7s 2ms/step - loss: 6.3686 - val_loss: 7.2169
Epoch 7/200
4006/4006 [==============================] - 8s 2ms/step - loss: 6.3463 - val_loss: 7.5044
Epoch 8/200
4006/4006 [==============================] - 7s 2ms/step - loss: 6.3112 - val_loss: 7.4303
Epoch 9/200
4006/4006 [==============================] - 7s 2ms/step - loss: 6.2685 - val_loss: 7.3892
Epoch 10/200
4006/4006 [=

4006/4006 [==============================] - 7s 2ms/step - loss: 0.4830 - val_loss: 8.9781
Epoch 80/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.4660 - val_loss: 9.0055
Epoch 81/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.4395 - val_loss: 8.9712
Epoch 82/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.4230 - val_loss: 8.9505
Epoch 83/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.3982 - val_loss: 9.0120
Epoch 84/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.3864 - val_loss: 8.9881
Epoch 85/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.3670 - val_loss: 8.9824
Epoch 86/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.3505 - val_loss: 8.9394
Epoch 87/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.3260 - val_loss: 9.0181
Epoch 88/200
4006/4006 [==============================] - 7s 2ms/step - loss:

4006/4006 [==============================] - 7s 2ms/step - loss: 0.0181 - val_loss: 9.3877
Epoch 158/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0183 - val_loss: 9.3784
Epoch 159/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0166 - val_loss: 9.3767
Epoch 160/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0151 - val_loss: 9.4345
Epoch 161/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0153 - val_loss: 9.3943
Epoch 162/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0143 - val_loss: 9.3726
Epoch 163/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0141 - val_loss: 9.3876
Epoch 164/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0132 - val_loss: 9.4110
Epoch 165/200
4006/4006 [==============================] - 7s 2ms/step - loss: 0.0130 - val_loss: 9.4235
Epoch 166/200
4006/4006 [==============================] - 7s 2ms/ste